In [1]:
pip install openprompt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.8 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.97
    Uninstalling sentencepiece-0.1.97:
      Successfully uninstalled sentencepiece-0.1.97
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 785.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from transformers import TrainingArguments, Trainer
import evaluate
from datasets import load_metric


In [4]:
model_type = "XLM-R" #@param ["SinBERT", "Bert", "XLM-R"]
technique = "humor" #@param ["humor", "hate speech", "sentiment"]
load_adapter = False #@param {type:"boolean"}
# train = True #@param {type:"boolean"}
unfreeze_model = False #@param {type:"boolean"}
save_adapter = False #@param {type:"boolean"}
oversample_dataset = False #@param {type:"boolean"}
lang_adapter_setting = "none" #@param ["none", "stack", "parallel"]
random_state = 42 #@param
adapter_config = "pfeiffer" #@param ["houlsby", "pfeiffer"]
over_sampling_technique = "ROS" #@param ["", "ROS","ADASYN", "SMOTE", "BorderlineSMOTE"]
sampling_strategy = "1:0.25:0.25" #@param [] {allow-input: true} 
## eg: 1:0.25:0.25 for hate | 0.5 for humor | 1:1:1:1 or 0.5:1:0.5:0.5 or 0.25:1:0.25:0.25 for sentiment

In [5]:
from openprompt.data_utils.text_classification_dataset import PROCESSORS
from datasets import load_dataset


In [6]:
dataset_path="/kaggle/input/cmcs-dataset-task/ompleted_draft - ompleted_draft.csv"
all_data = pd.read_csv(dataset_path)

if (technique == "humor"):
  all_data = all_data[['Sentence', 'Humor']]
elif (technique == "hate speech"):
  all_data = all_data[['Sentence', 'Hate_speech']]
else:
  all_data = all_data[['Sentence', 'Sentiment']]

all_data.columns = ['Sentence', 'Label']
all_data['Label'], uniq = pd.factorize(all_data['Label'])

X = all_data['Sentence'].values.tolist()
y = all_data['Label'].values.tolist()

In [7]:
all_data.groupby("Label").count()


,Sentence
Label,
0,12213
1,1305


In [8]:
uniq

Index(['Non-humorous', 'Humorous'], dtype='object')

In [9]:
if (technique == 'humor'):
    num_labels=2
    id2label={ 0: "Non-humorous", 1: "Humorous"}
elif (technique == 'hate speech'):
    num_labels=3
    id2label={ 0: "Not offensive", 1: "Hate-Inducing", 2: "Abusive"}
else:
    num_labels=4
    id2label={ 0: "Negative", 1: "Neutral", 2: "Positive", 3:"Conflict"}

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = random_state)

In [11]:
examples=[]
from openprompt.data_utils import InputExample
for i in range(len(X_train)):
  examples.append(InputExample(
        guid = i,
        text_a =X_train[i],
        label=y_train[i]
    ))

In [12]:

from transformers.models.auto.tokenization_auto import tokenizer_class_from_name

from openprompt.plms.utils import TokenizerWrapper
from typing import List, Dict
from collections import defaultdict

class MLMTokenizerWrapper(TokenizerWrapper):
    add_input_keys = ['input_ids', 'attention_mask', 'token_type_ids']

    @property
    def mask_token(self):
        return self.tokenizer.mask_token

    @property
    def mask_token_ids(self):
        return self.tokenizer.mask_token_id

    @property
    def num_special_tokens_to_add(self):
        if not hasattr(self, '_num_specials'):
            self._num_specials = self.tokenizer.num_special_tokens_to_add()
        return self._num_specials

    def tokenize_one_example(self, wrapped_example, teacher_forcing):
        ''' # TODO doesn't consider the situation that input has two parts
        '''

        wrapped_example, others = wrapped_example

        # for some dataset like SuperGLUE.COPA, the answer requires prediction an span of
        # the input. Or in generation tasks, we need to generate a piece of target_text.
        # In these case, it tokenized to the encoded_tgt_text for future use.
        encoded_tgt_text = []
        if 'tgt_text' in others:
            tgt_text = others['tgt_text']
            if isinstance(tgt_text, str):
                tgt_text = [tgt_text]
            for t in tgt_text:
                encoded_tgt_text.append(self.tokenizer.encode(t, add_special_tokens=False))


        mask_id = 0 # the i-th the mask token in the template.

        encoder_inputs = defaultdict(list)
        for piece in wrapped_example:
            if piece['loss_ids']==1:
                if teacher_forcing: # fill the mask with the tgt task
                    raise RuntimeError("Masked Language Model can't perform teacher forcing training!")
                else:
                    encode_text = [self.mask_token_ids]
                mask_id += 1

            if piece['text'] in self.special_tokens_maps.keys():
                to_replace = self.special_tokens_maps[piece['text']]
                if to_replace is not None:
                    piece['text'] = to_replace
                else:
                    raise KeyError("This tokenizer doesn't specify {} token.".format(piece['text']))

            if 'soft_token_ids' in piece and piece['soft_token_ids']!=0:
                encode_text = [0] # can be replace by any token, since these token will use their own embeddings
            else:
                encode_text = self.tokenizer.encode(piece['text'], add_special_tokens=False)

            encoding_length = len(encode_text)
            encoder_inputs['input_ids'].append(encode_text)
            for key in piece:
                if key not in ['text']:
                    encoder_inputs[key].append([piece[key]]*encoding_length)

        encoder_inputs = self.truncate(encoder_inputs=encoder_inputs)
        # delete shortenable ids
        encoder_inputs.pop("shortenable_ids")
        encoder_inputs = self.concate_parts(input_dict=encoder_inputs)
        encoder_inputs = self.add_special_tokens(encoder_inputs=encoder_inputs)
        # create special input ids
        encoder_inputs['attention_mask'] = [1] *len(encoder_inputs['input_ids'])
        if self.create_token_type_ids:
            encoder_inputs['token_type_ids'] = [0] *len(encoder_inputs['input_ids'])
        # padding
        encoder_inputs = self.padding(input_dict=encoder_inputs, max_len=self.max_seq_length, pad_id_for_inputs=self.tokenizer.pad_token_id)


        if len(encoded_tgt_text) > 0:
            encoder_inputs = {**encoder_inputs, "encoded_tgt_text": encoded_tgt_text}# convert defaultdict to dict
        else:
            encoder_inputs = {**encoder_inputs}
        return encoder_inputs

In [13]:
from statistics import mode
from typing import List, Optional
from transformers.modeling_utils import PreTrainedModel
from transformers.tokenization_utils import PreTrainedTokenizer
from transformers import BertConfig, BertTokenizer, BertModel, BertForMaskedLM, \
                         RobertaConfig, RobertaTokenizer, RobertaModel, RobertaForMaskedLM, \
                         XLMRobertaConfig, XLMRobertaTokenizer, XLMRobertaModel, XLMRobertaForMaskedLM
from collections import namedtuple
from yacs.config import CfgNode

from openprompt.utils.logging import logger


ModelClass = namedtuple("ModelClass", ('config', 'tokenizer', 'model','wrapper'))

_MODEL_CLASSES = {
    'bert': ModelClass(**{
        'config': BertConfig,
        'tokenizer': BertTokenizer,
        'model':BertForMaskedLM,
        'wrapper': MLMTokenizerWrapper,
    }),
    'roberta': ModelClass(**{
        'config': RobertaConfig,
        'tokenizer': RobertaTokenizer,
        'model':RobertaForMaskedLM,
        'wrapper': MLMTokenizerWrapper
    }),
    'xlm': ModelClass(**{
        'config': XLMRobertaConfig,
        'tokenizer': XLMRobertaTokenizer,
        'model': XLMRobertaForMaskedLM,
        'wrapper': MLMTokenizerWrapper
    }),
}


def get_model_class(plm_type: str):
    return _MODEL_CLASSES[plm_type]


def load_plm(model_name, model_path, specials_to_add = None):
    r"""A plm loader using a global config.
    It will load the model, tokenizer, and config simulatenously.

    Args:
        config (:obj:`CfgNode`): The global config from the CfgNode.

    Returns:
        :obj:`PreTrainedModel`: The pretrained model.
        :obj:`tokenizer`: The pretrained tokenizer.
        :obj:`model_config`: The config of the pretrained model.
        :obj:`wrapper`: The wrapper class of this plm.
    """
    model_class = get_model_class(plm_type = model_name)
    model_config = model_class.config.from_pretrained(model_path)
    # you can change huggingface model_config here
    # if 't5'  in model_name: # remove dropout according to PPT~\ref{}
    #     model_config.dropout_rate = 0.0
    if 'gpt' in model_name: # add pad token for gpt
        specials_to_add = ["<pad>"]
        # model_config.attn_pdrop = 0.0
        # model_config.resid_pdrop = 0.0
        # model_config.embd_pdrop = 0.0
    model = model_class.model.from_pretrained(model_path, config=model_config)
    tokenizer = model_class.tokenizer.from_pretrained(model_path)
    wrapper = model_class.wrapper


    model, tokenizer = add_special_tokens(model, tokenizer, specials_to_add=specials_to_add)

    if 'opt' in model_name:
        tokenizer.add_bos_token=False
    return model, tokenizer, model_config, wrapper




def load_plm_from_config(config: CfgNode):
    r"""A plm loader using a global config.
    It will load the model, tokenizer, and config simulatenously.

    Args:
        config (:obj:`CfgNode`): The global config from the CfgNode.

    Returns:
        :obj:`PreTrainedModel`: The pretrained model.
        :obj:`tokenizer`: The pretrained tokenizer.
        :obj:`model_config`: The config of the pretrained model.
        :obj:`model_config`: The wrapper class of this plm.
    """
    plm_config = config.plm
    model_class = get_model_class(plm_type = plm_config.model_name)
    model_config = model_class.config.from_pretrained(plm_config.model_path)
    # you can change huggingface model_config here
    # if 't5'  in plm_config.model_name: # remove dropout according to PPT~\ref{}
    #     model_config.dropout_rate = 0.0
    if 'gpt' in plm_config.model_name: # add pad token for gpt
        if "<pad>" not in config.plm.specials_to_add:
            config.plm.specials_to_add.append("<pad>")
    model = model_class.model.from_pretrained(plm_config.model_path, config=model_config)
    tokenizer = model_class.tokenizer.from_pretrained(plm_config.model_path)
    wrapper = model_class.wrapper
    model, tokenizer = add_special_tokens(model, tokenizer, specials_to_add=config.plm.specials_to_add)
    return model, tokenizer, model_config, wrapper

def add_special_tokens(model: PreTrainedModel,
                       tokenizer: PreTrainedTokenizer,
                       specials_to_add: Optional[List[str]] = None):
    r"""add the special_tokens to tokenizer if the special token
    is not in the tokenizer.

    Args:
        model (:obj:`PreTrainedModel`): The pretrained model to resize embedding
                after adding special tokens.
        tokenizer (:obj:`PreTrainedTokenizer`): The pretrained tokenizer to add special tokens.
        specials_to_add: (:obj:`List[str]`, optional): The special tokens to be added. Defaults to pad token.

    Returns:
        The resized model, The tokenizer with the added special tokens.

    """
    if specials_to_add is None:
        return model, tokenizer
    for token in specials_to_add:
        if "pad" in token.lower():
            if tokenizer.pad_token is None:
                tokenizer.add_special_tokens({'pad_token': token})
                model.resize_token_embeddings(len(tokenizer))
                logger.info("pad token is None, set to id {}".format(tokenizer.pad_token_id))
    return model, tokenizer

In [14]:
plm, tokenizer, model_config, WrapperClass =load_plm("xlm", "xlm-roberta-base")

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

In [15]:
from openprompt.prompts import MixedTemplate
# promptTemplate = MixedTemplate(
#     model=plm,
#     text = '{"placeholder": "text_a"} {"soft": "It"} {"soft": "was"} {"mask"}.',
#     tokenizer = tokenizer,
# )

from openprompt.prompts import ManualTemplate
promptTemplate = ManualTemplate(
    text = '{"placeholder": "text_a"}. It was {"mask"}.',
    tokenizer = tokenizer,
    
)

In [16]:
classes = [ 
    "0",
    "1",
]

In [17]:
from openprompt.prompts import SoftVerbalizer
promptVerbalizer = SoftVerbalizer(tokenizer, plm, num_classes=2)

In [18]:
from openprompt import PromptForClassification
promptModel = PromptForClassification(
    template = promptTemplate,
    plm = plm,
    verbalizer = promptVerbalizer,
)

In [19]:

MAX_LEN = 128

from openprompt import PromptDataLoader
data_loader = PromptDataLoader(
    dataset = examples,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
    batch_size=1,
    max_length=MAX_LEN,
    truncation=True,
    padding="max_length"
)

tokenizing: 12166it [00:08, 1421.83it/s]


In [20]:
import torch
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
# optimizer_grouped_parameters = [
#     {'params': [p for n,p in promptModel.template.named_parameters() if "raw_embedding" not in n]}
# ]
# optimizer = AdamW(optimizer_grouped_parameters, lr=1e-3)

optimizer_grouped_parameters1 = [
    {'params': promptModel.verbalizer.group_parameters_1, "lr":3e-5},
    {'params': promptModel.verbalizer.group_parameters_2, "lr":3e-4},
]

optimizer1 = AdamW(optimizer_grouped_parameters1)


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [21]:
promptModel=promptModel.cuda()
for epoch in range(2):
    tot_loss = 0
    for step, inputs in enumerate(data_loader):
        inputs = inputs.cuda()
        logits = promptModel(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
#         optimizer.step()
#         optimizer.zero_grad()
        optimizer1.step()
        optimizer1.zero_grad()
        print(tot_loss/(step+1))

1.0509833097457886
0.7478781342506409
0.8653489351272583
0.8931309282779694
0.8675777316093445
0.8291637897491455
0.8247788293021066
0.7950647696852684
0.8161069485876296
0.773958894610405
0.7361259893937544
0.701290637254715
0.6677715870050284
0.641910827585629
0.6250840286413829
0.601497215218842
0.5790430035661248
0.554446268412802
0.5331263683344188
0.5167356312274933
0.4995783475183305
0.48287036744031037
0.46651905256768933
0.4510610019788146
0.43688044816255567
0.4237206747325567
0.41000652671964083
0.47779289659644875
0.462786671930346
0.44875240909556546
0.43527907277307204
0.42322749737650156
0.4111239136845777
0.40033362488098007
0.460589697850602
0.4491336024883721
0.5020491635879955
0.49021261577543457
0.47894664557698446
0.4681638159789145
0.45893209818296316
0.4500805690352406
0.4425151087343693
0.4345930109477856
0.42642717419399156
0.47409773966216523
0.5097989128466617
0.5008117327621827
0.49281717874870007
0.48566083021461964
0.4774162105339415
0.4699925599046625
0.4

In [22]:
examples_test=[]
from openprompt.data_utils import InputExample
for i in range(len(X_test)):
  examples_test.append(InputExample(
        guid = i,
        text_a =X_test[i],
        label=y_test[i]
    ))
  

In [23]:
data_loader = PromptDataLoader(
    dataset = examples_test,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
    batch_size=1,
    max_length=MAX_LEN,
    truncation=True,
    padding="max_length"
)

tokenizing: 1352it [00:00, 1438.02it/s]


In [24]:
import torch
torch.cuda.empty_cache()
allpreds = []
alllabels = []
promptModel=promptModel.cuda()
for step, inputs in enumerate(data_loader):
    inputs=inputs.cuda();
    logits = promptModel(inputs)    
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())


In [25]:

def compute_metrics(allpreds,alllabels):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("f1")
    metric4 = load_metric("accuracy")
    
    predictions, labels = allpreds,alllabels
    precision = metric1.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]
    macro_precision = metric1.compute(predictions=predictions, references=labels, average="macro")["precision"]
    macro_recall = metric2.compute(predictions=predictions, references=labels, average="macro")["recall"]
    macro_f1 = metric3.compute(predictions=predictions, references=labels, average="macro")["f1"]
    return {"accuracy":accuracy, "precision": precision, "recall": recall, "f1": f1, "macro_precision": macro_precision, "macro_recall": macro_recall, "macro_f1": macro_f1}

In [26]:
compute_metrics(allpreds,alllabels)

{'accuracy': 0.9038461538461539,
 'precision': 0.8991128041287786,
 'recall': 0.9038461538461539,
 'f1': 0.9013563286406194,
 'macro_precision': 0.6766453674121405,
 'macro_recall': 0.6592453917050691,
 'macro_f1': 0.6673127593203912}

In [27]:
c=0
for i,j in zip(allpreds, alllabels):
  if(i!=j):
    print(c,X_test[c],"---->","predicted :", uniq[i]," actual :",  uniq[j])

  c+=1

7 4G Signal deehan okkotama kalin 😕 ----> predicted : Humorous  actual : Non-humorous
20 Idamath loan Gedarath loan Car ekath loan Lakshayak hambu karanakota Laksha ekahamaraka withara wiyadam. ----> predicted : Non-humorous  actual : Humorous
34 නිකන් දීපන් බොල...අපෙන් ගන්න සල්ලි මදිද 😂😂😂 ----> predicted : Non-humorous  actual : Humorous
49 King kong 😂🔫 ----> predicted : Non-humorous  actual : Humorous
51 යකො 138 අරගෙන තව daly reantal එකත් ගන්නව හුරගෙනම කාපියව් ----> predicted : Humorous  actual : Non-humorous
58 no signl no whtapp 😂 ----> predicted : Non-humorous  actual : Humorous
60 Grade 7 maths... ha ha ----> predicted : Non-humorous  actual : Humorous
73 Data adu kala witharai package eka wenas wenne ne lol ----> predicted : Non-humorous  actual : Humorous
77 සිම් එක පොලොවෙ ගහනවා දැනගනියව් ----> predicted : Humorous  actual : Non-humorous
84 I think I just broke your AI based chatbot 😞 ----> predicted : Non-humorous  actual : Humorous
88 ටිප්ස් නෑ පුතා... ----> predicted : Humor

In [28]:
c=0
for i,j in zip(allpreds, alllabels):
  if(i==j):
    print(c,X_test[c],"---->","predicted :", uniq[i]," actual :",  uniq[j])
    
  c+=1

0 Well done chamod de silva.... ----> predicted : Non-humorous  actual : Non-humorous
1 aka dawas kiyak thiyenawada? ----> predicted : Non-humorous  actual : Non-humorous
2 Data iwara unama kisima alert ekak nathuwa sallith ekka kapagena yanawa ammo dialog hodai 4G kiyala gattata mata oni! ----> predicted : Non-humorous  actual : Non-humorous
3 Connection is very slow ----> predicted : Non-humorous  actual : Non-humorous
4 Wal baduwa gayesha perera ----> predicted : Non-humorous  actual : Non-humorous
5 4G sim ekak ganne kohomada? ----> predicted : Non-humorous  actual : Non-humorous
6 apita signal full locatin eke idala 200m wath na tower ekata ----> predicted : Non-humorous  actual : Non-humorous
8 තාමත් නෑ ----> predicted : Non-humorous  actual : Non-humorous
9 කොත්මලේට රෑට සිග්නල් දෙන්නේ කවද්ද.. ----> predicted : Non-humorous  actual : Non-humorous
10 3. ආනන්ද ස්වාමින් වහන්සේ ----> predicted : Non-humorous  actual : Non-humorous
11 She finds cooking is a meditation n always enterta